In [23]:
import pandas as pd
import numpy as np
from  sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [24]:
#load the trained model
model=load_model('model.h5')

#load encoder and scalar
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo=pickle.load(file)

with open('scalar.pkl','rb') as file:
    scalar=pickle.load(file)

In [25]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [26]:
geo_encoded=one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\mahakalkar\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [27]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [28]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [29]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1) #concanated the geography column 
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [30]:
print("Expected by scaler:", scalar.n_features_in_)
print("Input shape:", input_df.shape)

Expected by scaler: 12
Input shape: (1, 12)


In [31]:
input_scaled=scalar.transform(input_df)
input_scaled

c:\Users\mahakalkar\venv1\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


array([[ 6.00000000e+02,  1.00000000e+00,  4.00000000e+01,
         3.00000000e+00,  6.00000000e+04,  2.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  5.00000000e+04,
         1.00000000e+00,  7.10542736e-18, -4.26325641e-17]])

In [37]:
#prediction
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[1.]], dtype=float32)

In [38]:
prediction_prob=prediction[0][0]

In [39]:
prediction_prob

np.float32(1.0)

In [40]:
if prediction_prob > 0.5:
    print("Customer likely to exit")
else:
    print("Customer likely to stay")

Customer likely to exit
